In [20]:
import ee
import folium
import functools
import numpy as np
import pandas as pd

In [21]:
ee.Initialize()

# Load sample data
To start out, you will need a CSV file with one row for each sample. It must have columns titled `latitude`, `longitude`, and `sample_id`. We will use the coordinates to get more data about each sample location.


In [22]:
csv_path = 'PoURLocationsRound2.csv'

# Read the relevant columns from the CSV into a dataframe
samples = pd.read_csv(csv_path, usecols=['sample_id', 'longitude', 'latitude'])

# Display the shape of the data we read in and its first few rows
print("Data shape:", samples.shape)
print(samples.head())

# Define a Feature for each 100-meter-radius sample area
sample_areas = []
for sample in samples.itertuples():
    # Store the important data as properties of the feature
    sample_areas.append(
        ee.Feature(ee.Geometry.Point(sample.longitude, sample.latitude).buffer(100))
        .set('name', sample.sample_id)
        .set('longitude', sample.longitude)
        .set('latitude', sample.latitude))

# Define a FeatureCollection containing all the sample areas
sample_areas = ee.FeatureCollection(sample_areas)

Data shape: (153, 3)
  sample_id   longitude   latitude
0    AS_DB1 -118.166442  34.203437
1    AS_DB2 -118.166357  34.203842
2    AS_PN1 -118.166416  34.203148
3    AS_PN2 -118.166370  34.204008
4  K0561_A1 -118.166493  34.203237


# Define dataset classes
GIS datasets can be in either vector or raster format. In Google Earth Engine, raster data is represented by an `Image` or `ImageCollection` and vector data by a `FeatureCollection`. You can find datasets by searching in the [Earth Engine data catalog](https://developers.google.com/earth-engine/datasets). It's also possible to instantiate an Earth Engine object from data that you have locally if it's not available through the API.

To conceptualize the data it will be helpful to wrap these Earth Engine objects in our own classes. This way we can store information we need to use the dataset alongside it: the band or property to use and how to process and display the data.

## Raster datasets

In [23]:
class RasterDataset:
    
    def __init__(self, snippet, band, name=None, categorical=False, map_params={}):
        """
        Represent a Google Earth Engine raster dataset and the information we need to use it.
        
        snippet: The ee.Image or ee.ImageCollection object representing the desired dataset. 
            This is the snippet of code provided on the dataset's page in the Earth Engine catalog.
            e.g.: ee.Image('path') or ee.ImageCollection('path')
        band: The identifier of the desired image band e.g. 'bio01'
        name: The column name to display for the data gathered from this dataset
        categorical: Set to True if the data is not continuous. If True, takes the mode of pixels
            in the sample area rather than the median.
        map_params: Settings for how to display the data e.g. palette, min, max. 
            See param options: https://developers.google.com/earth-engine/api_docs#ee.data.getmapid
        """
        
        # self.name exists to be a common property between RasterDatasets and VectorDatasets
        self.name = name or band  # If a different name is not set, use the band name
        self.categorical = categorical
        self.map_params = map_params    
        
        # Select just the one band of interest from the image and rename it to self.name
        if isinstance(snippet, ee.ImageCollection):  
            # If it's an image collection, mosaic it into one image
            self.data = snippet.select([band], [self.name]).mosaic()
        else:
            self.data = snippet.select([band], [self.name])
        
        self.band = self.name  
        
    def get_sample_area_data(self, sample_area: ee.Feature) -> ee.Feature:
        """
        Add a new property to the input feature storing the value of self.data in the feature area
        
        param sample_area: ee.Feature with the geometry to sample over
        returns: input feature with a new property in the form 
            {self.band: mode (categorical) or median (continuous) of self.data 
                        over the feature geometry}
        """
        
        # Get the average distance from each point in the sample area to the data image
        distance = self.distance(sample_area.geometry())
        
        geometry = ee.Algorithms.If(
            distance,  # If the distance is not None,
            ee.Algorithms.If(
                distance.eq(0),  # If the distance is 0, the data covers the sample area
                sample_area.geometry(),
                sample_area.geometry().buffer(distance), # Otherwise, we need to buffer it
            ),
            # If the distance is None, the sample area is farther away from the data than the
            # range being searched by the distance kernel.
            # Continue with the sample area geometry, the reducer will return None.
            sample_area.geometry()
        )
        result = ee.Algorithms.If(
            self.categorical,  # If the data type is categorical,
            self.mode(geometry),  # Get the most common category in the sample area
            self.median(geometry)  # Otherwise it's continuous, so get the median
        )
    
        return sample_area.set(self.band, result)
    
    def distance(self, geometry: ee.Geometry) -> ee.Number:
        """
        Return the average distance from each point in the input geometry to the self.data image
        
        - This should be 0 in most cases, meaning the data covers the sample area.
        - If the sample area is within the search radius from the self.data image 
            (here set to 1000m), the distance will be returned.
        - If the sample area is outside the search radius, None is returned.
        
        This is intended to account for edges of datasets not lining up exactly with the 
        coastline, leaving some coastal sample sites just outside the self.data image.
        
        param geometry: ee.Geometry of the sample area of interest
        returns: ee.Number (0 or distance) or None
        """
        # Make an image representing the distance from any given point to the self.data image
        # This will be 0 everywhere that the self.data image covers
        distance_image = self.data.distance(
            ee.Kernel.euclidean(1000, 'meters'),
            False  # Do not exclude masked pixels
        ).select([self.band], ['distance'])  # Rename the band to 'distance'
        
        # Get the average distance from each point in the sample area to the nearest pixel of data
        distance = ee.Number(distance_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=geometry,
            scale=100,
            maxPixels=1e9
        ).get('distance'))
        
        return distance
    
    def mode(self, geometry: ee.Geometry) -> ee.Number:
        """
        Reduce the dataset over the sample area to get the most common value in that area.
        
        param geometry: ee.Geometry of the sample area of interest
        returns: mode of pixels within the geometry as an ee.Number
        """
        mode = self.data.reduceRegion(
            reducer=ee.Reducer.mode(),
            geometry=geometry,
            scale=100,
            maxPixels=1e9
        ).get(self.band)
        return mode
    
    def median(self, geometry: ee.Geometry) -> ee.Number:
        """
        Reduce the dataset over the sample area to get the median value in that area
        
        param geometry: ee.Geometry of the sample area of interest
        returns: median of pixels within the geometry as an ee.Number
        """
        median = self.data.reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=geometry,
            scale=100,
            maxPixels=1e9
        ).get(self.band)
        return median
        


## Vector datasets


In [24]:
class VectorDataset:
    
    def __init__(self, snippet, property: str, name=None, map_params={}):
        """
        Represent a Google Earth Engine vector dataset and the information we need to use it.
        
        snippet: The ee.FeatureCollection object representing the dataset.
            This is the 'Earth Engine snippet' displayed on the dataset's page in the Earth 
            Engine catalog. e.g.: ee.FeatureCollection('EPA/Ecoregions/2013/L3')
        property: The key of the desired feature property
        name: The column name to display for the data gathered from this dataset.
        map_params: Settings for how to display the data e.g. palette, min, max
        """
        self.data = snippet
        self.property = property
        self.name = name or property
        self.map_params = map_params
        
    # Note: This function is an argument to map(). Arguments to map() cannot print anything
    # or call getInfo(). Doing so results in an EEException: ValueNode empty
    # source: https://gis.stackexchange.com/questions/345598/mapping-simp>le-function-to-print-date-and-time-stamp-on-google-earth-engine-pyth
    def get_sample_area_data(self, sample_area: ee.Feature) -> str:
        """
        Return the value from the dataset to assign to the sample area.
        """
        # Get a FeatureCollection storing the overlaps between the sample area and the dataset
        overlaps = self.data.filterBounds(sample_area.geometry()).map(
            lambda feature: feature.intersection(sample_area.geometry())
        )

        result = ee.Algorithms.If(
            # If there is exactly 1 overlapping dataset feature, return its value
            overlaps.size().eq(1),
            sample_area.set(self.name, overlaps.first().get(self.property)),

            ee.Algorithms.If(
                # If there are 0 overlapping dataset features, return the value of the closest one
                overlaps.size().eq(0),
                sample_area.set(self.name, 
                                self.get_nearest_feature(sample_area).get(self.property)),

                # Otherwise, there must be >1 features overlapping the sample area
                # Return the value of the one with the largest overlap
                sample_area.set(self.name, 
                                self.get_predominant_feature(overlaps).get(self.property))
            )
        )

        return result

    def get_nearest_feature(self, sample_area: ee.Feature) -> str:
        """
        To be used when the sample area doesn't overlap the dataset at all.
        
        Get the dataset feature that is nearest to the sample area, and
        return the value of its dataset.property.
        
        param sample_area: ee.Feature representing the sample area of interest
        """

        # Define a filter to get all dataset features within 10000 meters of the sample area
        spatialFilter = ee.Filter.withinDistance(
            distance=10000,
            leftField='.geo',
            rightField='.geo',
            maxError=10
        )
        # Define a join that will return only the 'best' (nearest) match
        saveBestJoin = ee.Join.saveBest(
          matchKey='closestFeature',
          measureKey='distance'
        )
        # Apply the join, using the distance filter to define match quality
        # Get the only feature in the resulting FeatureCollection
        result = ee.Feature(saveBestJoin.apply(
            ee.FeatureCollection(sample_area),
            self.data,
            spatialFilter
        ).first())

        # Return the closest dataset feature
        return ee.Feature(result.get('closestFeature'))
    
    def get_predominant_feature(self, overlaps: ee.FeatureCollection) -> str:
        """
        To be used when the sample area overlaps more than one dataset feature.
        
        Return the value of 'property' for the largest overlap.
        """
        # Add 'area' as a property to each feature. This is the area in square meters 
        # of the intersection of the ecoregion feature and the sample area.
        overlaps = overlaps.map(
            lambda feature: feature.set({'area': feature.geometry().area()}))

        # Find the maximum area among all the overlaps
        max_area = overlaps.aggregate_max('area')

        # Return the overlap with the largest area
        return ee.Feature(overlaps.filter(ee.Filter.gte('area', max_area)).first())
    

## Instantiate your datasets
Instantiate your chosen Earth Engine datasets in the list below.

### BioClim data
The [BioClim dataset](https://developers.google.com/earth-engine/datasets/catalog/WORLDCLIM_V1_BIO) has 19 bands representing different measures of temperature and precipitation to a resolution of 30 arc seconds (1 kilometer).

Some of the bands have had their values multiplied by a factor of 10 or 100 in order to store the data in integer format. To get the actual values we need to divide each pixel in the image by that factor. We can do this with the [`ee.Image.divide`](https://developers.google.com/earth-engine/api_docs#ee.image.divide) function, which we apply to the `image` argument to our dataset constructors. In a similar way, any snippet which evaluates to an `ee.Image` or `ee.ImageCollection` object may be used as the input to the `RasterDataset` constructor. This way we can preprocess the dataset if desired, before extracting values.

In [25]:
bioclim = [
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)),
        band='bio01'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)),
        band='bio02'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio03'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(100)), 
        band='bio04'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio05'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio06'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio07'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio08'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio09'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio10'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO').divide(ee.Image(10)), 
        band='bio11'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'),
        band='bio12'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio13'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio14'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'),
        band='bio15'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'),
        band='bio16'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio17'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio18'
    ),
    RasterDataset(
        snippet=ee.Image('WORLDCLIM/V1/BIO'), 
        band='bio19'
    )
]

### Level III Ecoregions
The [EPA Level III Ecoregions of the Conterminous United States](https://developers.google.com/earth-engine/datasets/catalog/EPA_Ecoregions_2013_L3?hl=en) is our only dataset in vector format. We instantiate it as a `VectorDataset` and specify the property we want to extract from each feature, `us_l3name`.

In [26]:
ecoregions = [
    VectorDataset(
        snippet=ee.FeatureCollection('EPA/Ecoregions/2013/L3'),
        property='us_l3name',
        map_params={'min': 0, 'max': 10}
    )
]

### Soil properties
Soil data is available at 250 meter resolution from [OpenLandMap datasets](https://developers.google.com/earth-engine/datasets/tags/openlandmap). We are selecting the values at surface level (band `b0`) but other bands correspond to depths of 10, 30, 60, 100, and 200 centimeters. Three of these datasets have been scaled up (shown in the 'Scale' column in the 'Bands' tab on the dataset pages) and so we divide them to get the true values.

Since these datasets have band names which aren't self explanatory, for clarity we give them an alternate `name` which will be used in the output.


In [27]:
soil = [
    RasterDataset(  # Soil taxonomy great groups, a classification number from 0 to 433
        snippet=ee.Image('OpenLandMap/SOL/SOL_GRTGROUP_USDA-SOILTAX_C/v01'),
        band='grtgroup',
        categorical=True
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_PH-H2O_USDA-4C1A2A_M/v02').divide(ee.Image(10)),
        band='b0',
        name='soil pH in H20'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02').divide(ee.Image(5)),
        band='b0',
        name='soil organic carbon'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b0',
        name='soil sand'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02'),
        band='b0',
        name='soil clay'
    ),
    RasterDataset(
        snippet=ee.Image('OpenLandMap/SOL/SOL_BULKDENS-FINEEARTH_USDA-4A1H_M/v02').divide(ee.Image(10)),
        band='b0',
        name='soil bulk density'
    )
]

### Terrain
Elevation data for the United States is available from the [USGS National Elevation dataset](https://developers.google.com/earth-engine/datasets/catalog/USGS_NED) at a resolution of 10 meters. Slope and aspect datasets are not available, but they are a direct function of elevation. Here we preprocess the elevation image using the [`ee.Terrain`](https://developers.google.com/earth-engine/api_docs#ee.terrain.aspect) functions to derive slope and aspect images.

In [28]:
terrain = [
    RasterDataset(
        snippet=ee.Image('USGS/NED'),
        band='elevation'
    ),
    RasterDataset(
        snippet=ee.Terrain.slope(ee.Image('USGS/NED')),
        band='slope'
    ),
    RasterDataset(
        snippet=ee.Terrain.aspect(ee.Image('USGS/NED')),
        band='aspect'
    ),
]

# Human Influence Index
Data for the CSP gHM: Global Human Modification is derived from the [global Human Modification dataset](https://developers.google.com/earth-engine/datasets/catalog/CSP_HM_GlobalHumanModification). Which provides a cumulative measure of human modification of terrestrial lands globally at 1 square-kilometer resolution. The gHM values range from 0.0-1.0 and are calculated by estimating the proportion of a given location (pixel) that is modified, the estimated intensity of modification associated with a given type of human modification or "stressor". 5 major anthropogenic stressors circa 2016 were mapped using 13 individual datasets:
1. human settlement (population density, built-up areas)
2. agriculture (cropland, livestock)
3. transportation (major, minor, and two-track roads; railroads)
4. mining and energy production
5. electrical infrastructure (power lines, nighttime lights)


In [29]:
HII = [
    RasterDataset(
        snippet=ee.ImageCollection('CSP/HM/GlobalHumanModification'),
        band='gHM'
    ),
]
       

### Landsat 8
Datasets derived from composite Landsat 8 satellite imagery, taken about every 2 weeks at a 30 meter resolution, include normalized difference vegetation index (NDVI), enhanced vegetation index (EVI), normalized burn ratio thermal (NBRT), and greenest pixel.

This is our first `ImageCollection` data. An `ImageCollection` represents a series of images over time. We will use the `ee.ImageCollection.filterDate` function to filter the series to our desired date range. The filtered collection will be mosaiced together to form a single image. 

* If you are interested in data from a specific point in time, you can set the date range accordingly, but be aware that data from a single day might not cover the entire area of interest. You will need to experiment with different ranges and find a minimal range that has complete coverage.

* If the timestamp is not important to your data, you should still choose an appropriate date range, though it may be broader. If you do not specify a date range, overlapping images will be composited by their order in the collection, which could give you a mix of old and new data.

In [30]:
landsat = [
    RasterDataset(
        snippet=ee.ImageCollection('LANDSAT/LC08/C01/T1_32DAY_NDVI').filterDate(
            '2020-10-27', '2021-02-21'
        ),
        band='NDVI',
        map_params={'min': 0, 'max': 1, 'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]}
    ),
    RasterDataset(
        snippet=ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_EVI').filterDate(
            '2020-10-27', '2021-02-21'
        ),
        band='EVI'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NBRT').filterDate(
            '2020-10-27', '2021-02-21'
        ),
        band='NBRT'
    ),
    RasterDataset(
        snippet=ee.Image('LANDSAT/LC8_L1T_ANNUAL_GREENEST_TOA/2017'),
        band='greenness'
    ),
]

### Land cover
The [USGS National Land Cover Database](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD?hl=en) includes the landcover classification as well as continuous variables like percent impervious surface and percent tree cover at a 30 meter resolution.

Here we see another kind of filter we can use to preprocess the data. Not all the bands we want are available in every image in the collection. We choose the most recent image, NLCD2016, by filtering on the image index.

Note that landcover classification is a categorical variable and the others are continuous. We use `categorical=True` for landcover so that sample site values will be calculated by taking the mode rather than the median, which doesn't make sense for categorical data.

In [31]:
landcover = [
    RasterDataset(
        snippet=ee.ImageCollection('USGS/NLCD').filterMetadata(
            'system:index',
            'equals',
            'NLCD2011'
        ),
        band='landcover',
        categorical=True,
        map_params={'min': 0, 'max': 95}
    ),
    RasterDataset(
        snippet=ee.ImageCollection('USGS/NLCD').filterMetadata(
            'system:index',
            'equals',
            'NLCD2011'
        ),
        band='impervious'
    ),
    RasterDataset(
        snippet=ee.ImageCollection('USGS/NLCD').filterMetadata(
            'system:index',
            'equals',
            'NLCD2011'
        ),
        band='percent_tree_cover'
    )
]

### Sentinel 2
The Sentinel 2 satellites https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2 continuously take multi-spectral imagery of the Earth, covering a given location once every 5 days on average since 2015. Resolution is 10, 20, or 60 meters depending on the band.

This data needs a few preprocessing steps. Because this dataset is huge with tens of thousands of images, using all of them would be very slow. We filter to a date range and filter out images with more than 20% cloudy pixels. We define a method that filters the set to only include images covering California. We also define a cloud masking function, which makes pixels transparent if the QA60 band indicates they have cloud or cirrus cover. And we reduce the `ImageCollection` to a single `Image` by using the `median` method, which sets each pixel to the median value from all images covering that point.

In [32]:
def mask_clouds(image: ee.Image) -> ee.Image:
    '''Return the image with clouds masked'''
    
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = int(2 ** 10)
    cirrusBitMask = int(2 ** 11)

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    image = image.updateMask(mask)
    
    return image

def filter_california(collection: ee.ImageCollection) -> ee.ImageCollection:
    '''Filter the collection to just those images overlapping California'''
    # Define a rectangular area just big enough to contain all of California
    california = ee.Geometry.Polygon([
        ee.Geometry.Point(-125, 32.5),
        ee.Geometry.Point(-125, 42),
        ee.Geometry.Point(-114, 42),
        ee.Geometry.Point(-114, 32.5)
    ])
    # Return a collection of just those images that are within California
    return collection.filterBounds(california)


Next we preprocess the collection. We filter the collection to include just those images that are in California and within a date range (adjust this as appropriate for your data). We apply the cloud masking function to each image, then divide each image by 10000 to correct the scale. This preprocessed collection is used as the input to the `RasterDataset` constructors.

In [33]:

sentinel_preprocessed = filter_california(
                            ee.ImageCollection("COPERNICUS/S2")
                        ).filterDate(
                            '2020-10-27',
                            '2021-02-21'
                        ).filter(
                            ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)
                        ).map(
                            lambda img: mask_clouds(img).divide(
                                ee.Image(10000)
                            )
                        ).median()

sentinel = [
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B1',
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B2',
        map_params={'min': 0, 'max': 1}
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B3',
        map_params={'min': 0, 'max': 1}
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B4',
        map_params={'min': 0, 'max': 1}
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B5'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B6'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B7'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B8'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B8A'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B9'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B10'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B11'
    ),
    RasterDataset(
        snippet=sentinel_preprocessed,
        band='B12'
    ),
]

### Concatenate the lists of datasets

In [34]:
# Define the list of datasets from which to retrieve data
datasets = bioclim + ecoregions + soil + terrain + HII + landsat + landcover + sentinel

In [35]:
# Here is an extra function for reprojecting an Image to a different scale. 
# Might be useful for something, not sure what effect this has exactly

def reproject(image: ee.Image, scale: int) -> ee.Image:
    '''
    Resample and reproject an image to a different resolution.
    
    param image: Image to reproject
    param scale: Desired pixel width in meters
    '''
    return image.resample('bilinear').reproject(  # Use bilinear interpolation method
        crs=image.projection().crs(),  # Keep the same map projection
        scale=scale  # Change the scale
    )




# Map the data
Next we define a mapping class and some methods to help visualize the data. This will help with understanding what we're doing, and also make it easy to visually verify that the results make sense. The JavaScript version of the Earth Engine API provides a Map class that makes it easy to do this, but the Python API doesn't have that feature. I'm implementing it using Folium, a Python library for creating Leaflet Javascript maps, as recommended [here](https://developers.google.com/earth-engine/python_install-colab.html#interactive_map).

In [36]:
# These two methods are going to be added as custom folium map methods. 
# This will make it easy to display vector and raster Earth Engine objects to a map.
def add_raster_layer(self, ee_image_object, vis_params, name):
    """
    Display an EE image (raster) on a folium map
    
    param ee_image_object: ee.Image to add to the map
    param vis_params: map visualization parameters e.g. min, max, palette
    param name: name to give the layer in the layer control panel display
    """
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data © Google Earth Engine",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

def add_vector_layer(self, coords, name):
    """
    Display an EE geometry (vector) on a folium map
    
    param ee_image_object: ee.Image to add to the map
    param vis_params: map visualization parameters e.g. min, max, palette
    param name: name to give the layer in the layer control panel display
    """
    # Reverse the order of the coordinates from (lng, lat) to (lat, lng)
    coords = np.array([np.flip(i) for i in coords])
    
    # Add the coordinates as a polygon layer in the map
    folium.vector_layers.Polygon(
        locations=coords,
        name=name,
        color='red',
        overlay=True,
        control=True,
        tooltip=name
    ).add_to(self)

Next we define the Map class, which makes use of the above methods to draw multiple datasets onto a folium map and display it.

In [37]:
class Map:
    
    def __init__(self):
        """Initialize a custom folium map"""
        # Add EE drawing methods to folium.
        folium.Map.add_raster_layer = add_raster_layer
        folium.Map.add_vector_layer = add_vector_layer
        
        # Centers the map on California. Adjust the center coordinates and zoom level for your data
        self.map = folium.Map(location=[35, -119], zoom_start=4, height=500)
        
    def add_polygon(self, coords: np.ndarray, name: str):
        """
        Add a polygon to the map
        
        param coords: array of (lng, lat) coordinate pairs outlining the polygon
        param name: name to give the polygon to display as a tooltip
        """
        self.map.add_vector_layer(coords, name)

    def add(self, dataset):
        """Add a dataset as a layer to an interactive map using folium"""

        # If the dataset is in vector format, we want to convert it to raster in order to 
        # display on the map more easily,
        if isinstance(dataset, VectorDataset):
            map_property = dataset.property
            
            # Non-numeric values can't be converted into raster format
            # We will have to arbitrarily assign numeric values to represent them
            if not isinstance(dataset.data.first().get(dataset.property), ee.Number):
                # Get an array of all existing values of the property
                values = ee.List(dataset.data.aggregate_array(dataset.property))
                # Use the index of each property in the array as an arbitrary numeric value
                # e.g. if your values were ['forest', 'desert', 'water'],
                # the new band would have values [0, 1, 2] respectively.
                dataset.data = dataset.data.map(
                    lambda feature: feature.set(
                        'as_number', values.indexOf(feature.get(dataset.property)))
                )
                # Do mapping based on the new numeric property
                map_property = 'as_number'
            
            # Convert to a raster image, turning values of the given property into pixel values
            image = dataset.data.reduceToImage(
                properties=[map_property], reducer=ee.Reducer.first())
            
        # If it's in raster format, we can use it as-is
        else:
            image = dataset.data
        
        self.map.add_raster_layer(
            image.updateMask(image.gt(0)), 
            dataset.map_params, 
            dataset.name)
        
    def get_coords(fc: ee.FeatureCollection) -> np.array:
        """
        Return a list of coordinate lists representing the geometries 
        of the features in the FeatureCollection.
        """
        coords = np.array(fc.iterate(
            lambda item, l: ee.List(l).add(item.geometry().coordinates()), 
            ee.List([])).getInfo())
        return coords
    
    def get_names(fc: ee.FeatureCollection) -> np.array:
        """
        Return a list of names extracted from the 'name' property of each 
        feature in the FeatureCollection.
        """
        names = np.array(fc.iterate(
            lambda item, l: ee.List(l).add(item.get('name')), 
            ee.List([])).getInfo())
        return names

    def display(self):
        """Display the map"""
        self.map.add_child(folium.LayerControl())  # Add a layer control panel to the map
        display(self.map)   # Display the map


# Run

In [38]:
# Define a map that will display all the data together, and add each sample area geometry to it
map = Map()
for polygon, name in zip(Map.get_coords(sample_areas), Map.get_names(sample_areas)):
    map.add_polygon(polygon, name=name)

# For each dataset,
for dataset in datasets:
    print(dataset.name)
    # Add a property to each sample area storing the value calculated from the dataset
    sample_areas = sample_areas.map(lambda feature: dataset.get_sample_area_data(feature))
    # Add the dataset as a layer to the map
    map.add(dataset)
    
map.display()  # Display the map

# Get the name of each dataset, which will be the column headers
dataset_names = [dataset.name for dataset in datasets]

# Export the sample area data to your Google Drive as a CSV
# Note: It may take a few minutes to show up.
task = ee.batch.Export.table.toDrive(
    collection=sample_areas, 
    description='eedata',  # The file will show up in your Drive with this name
    selectors=['name', 'latitude', 'longitude'] + dataset_names  # Don't output geometry property
)
task.start()

bio01
bio02
bio03
bio04
bio05
bio06
bio07
bio08
bio09
bio10
bio11
bio12
bio13
bio14
bio15
bio16
bio17
bio18
bio19
us_l3name
grtgroup
soil pH in H20
soil organic carbon
soil sand
soil clay
soil bulk density
elevation
slope
aspect
gHM
NDVI
EVI
NBRT
greenness
landcover
impervious
percent_tree_cover
B1
B2
B3
B4
B5
B6
B7
B8
B8A
B9
B10
B11
B12
